In [1]:
import quippy
import ase
from ase.calculators.calculator import all_changes

import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import os
import sys

In [4]:
sys.path.insert(0,"/home/es732/programs/function_modules/")
import mbx_functions

In [5]:
dirpath = os.getcwd()
print dirpath

/home/es732/programs/MBX/quip_interface/test


In [6]:
from collections import OrderedDict

In [7]:
d_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001]

In [8]:
version_list=["mbx_pbc","mbx_pbc_no_mww_pip","mbx_pbc_no_www_mww","mbx_pbc_no_ww_mw_www_mww",
              "mbx_pbc_no_ww_mw","mbx_pbc_no_ww","mbx_pbc_no_mw"]

In [9]:
dict_version_d_virial=OrderedDict([(version,
                                    OrderedDict([(d, [[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                                 for d in d_list]))
                                   for version in version_list])

In [10]:
dict_version_model_virial=OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                   for version in version_list])

In [11]:
def calculate_numerical_virial_diag(pot, in_atoms, d=1e-2):
    ## modified from the function calculate_numerical_stress() at ase/calculators/calculator.py 
    """Calculate numerical stress using finite difference."""

    atoms=in_atoms.copy()
    stress = np.zeros((3, 3), dtype=float)

    cell = atoms.cell.copy()
    V = atoms.get_volume()
    for i in range(3):
        x = np.eye(3)
        x[i, i] += d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eplus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        x[i, i] -= 2 * d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eminus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        stress[i, i] = (eplus - eminus) / (2 * d * V)
        x[i, i] += d
    atoms.set_cell(cell, scale_atoms=True)
    virial= (-stress*atoms.get_volume())
    return virial

In [12]:
this_root="/home/es732/programs/mbx_tests/single_point/pbc/clathrate/"

In [13]:
this_struct="phase_clathrate_2x2x2"
this_at_tmp=quippy.Atoms(this_root+"11.74290/"+this_struct+".xyz")
this_at_tmp.set_cell(np.eye(3)*50.)
dict_pot=mbx_functions.create_mbx_potential_variables(this_at_tmp)

In [14]:
this_at_tmp.lattice

FortranArray([[50.,  0.,  0.],
              [ 0., 50.,  0.],
              [ 0.,  0., 50.]])

## diagonal_virial=F, no PIPs

In [15]:
this_version=version_list[3]
print this_version

mbx_pbc_no_ww_mw_www_mww


In [16]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [17]:
at= this_at_tmp.copy()

In [18]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1073.80920385    21.69910986    20.51525099]
 [   21.69910986 -1077.35753592    28.61648701]
 [   20.51525099    28.61648701 -1089.69986688]]


In [19]:
np.savetxt("test.txt",dict_version_model_virial[this_version])

In [29]:
this_at_tmp.write("test.xyz")

In [20]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, full MBX

In [53]:
this_version=version_list[0]
print this_version

mbx_pbc


In [54]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [55]:
at= this_at_tmp.copy()

In [24]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-249.06631645   -8.67955405   -9.35353365]
 [  -8.67955405 -255.80913537   -9.74435778]
 [  -9.35353365   -9.74435778 -252.20803778]]


In [25]:
for tmp_d in [0.1,0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [56]:
for tmp_d in [0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no 3b PIPs

In [30]:
this_version=version_list[2]
print this_version

mbx_pbc_no_www_mww


In [31]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [32]:
at= this_at_tmp.copy()

In [33]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-266.05005826   -9.18689015   -7.97998036]
 [  -9.18689015 -271.57603558  -11.17364313]
 [  -7.97998036  -11.17364313 -268.5041102 ]]


In [34]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no 2b PIPs

In [35]:
this_version=version_list[4]
print this_version

mbx_pbc_no_ww_mw


In [36]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [37]:
at= this_at_tmp.copy()

In [38]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1056.82546205    22.20644597    19.1416977 ]
 [   22.20644597 -1061.59063572    30.04577237]
 [   19.1416977     30.04577237 -1073.40379446]]


In [39]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [40]:
for tmp_d in [0.00001,0.000001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no mw 2b PIPs

In [41]:
this_version=version_list[6]
print this_version

mbx_pbc_no_mw


In [42]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [43]:
at= this_at_tmp.copy()

In [44]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-284.45293315   -8.90231248   -9.12952282]
 [  -8.90231248 -291.71378904   -9.08627096]
 [  -9.12952282   -9.08627096 -288.83887944]]


In [45]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [46]:
for tmp_d in [0.00001,0.000001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [47]:
version_list

['mbx_pbc',
 'mbx_pbc_no_mww_pip',
 'mbx_pbc_no_www_mww',
 'mbx_pbc_no_ww_mw_www_mww',
 'mbx_pbc_no_ww_mw',
 'mbx_pbc_no_ww',
 'mbx_pbc_no_mw']

## Errors in derivative virial / Virial from model (only diagonal)

### full model

In [57]:
this_version="mbx_pbc"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]: #, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[0.00642574 0.00646207 0.00581412]
d = 0.001
[0.00912306 0.00904669 0.00839041]
d = 0.0001
[0.0091498  0.00907314 0.00841578]
d = 1e-05
[0.00915007 0.00907338 0.00841604]
d = 1e-06
[0.00914994 0.0090733  0.00841618]


### model without PIPs

In [27]:
this_version="mbx_pbc_no_ww_mw_www_mww"
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.00042727 -0.00041367 -0.00037826]
d = 0.001
[-4.28322649e-06 -4.10937794e-06 -3.80407531e-06]
d = 0.0001
[-4.32183075e-08 -4.13607599e-08 -3.79029714e-08]


### model without 2B PIPs

In [48]:
this_version="mbx_pbc_no_ww_mw"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.00041203 -0.00040184 -0.00037433]
d = 0.001
[-3.97920066e-06 -4.18341124e-06 -3.62223334e-06]
d = 0.0001
[-4.02749893e-08 -4.04159547e-08 -3.90183055e-08]
d = 1e-05
[-3.54045932e-10 -7.69515896e-10  7.33288738e-10]
d = 1e-06
[-2.41117416e-08 -7.81878271e-09  3.10878005e-08]


### model without 3B PIPs

In [49]:
this_version="mbx_pbc_no_www_mww"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[0.00592773 0.00601662 0.00542256]
d = 0.001
[0.00853919 0.00852151 0.00788022]
d = 0.0001
[0.00856569 0.00854637 0.00790502]
d = 1e-05
[nan nan nan]
d = 1e-06
[nan nan nan]


### model without 2B mw PIPs

In [52]:
this_version="mbx_pbc_no_mw"
for tmp_d in [0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[-0.00233483 -0.00223544 -0.00221574]
d = 0.001
[-2.28829745e-05 -2.28836205e-05 -2.16770921e-05]
d = 0.0001
[-2.31092921e-07 -2.16580897e-07 -2.20848923e-07]
d = 1e-05
[-2.99619241e-09 -9.61934959e-09 -1.36800386e-09]
d = 1e-06
[-1.16661874e-07 -8.18345542e-08  1.16938165e-07]
